# Advanced LangChain Features
In the [Getting Started notebook](./getting_started.ipynb), we covered an introduction to LangChain and demonstrated how to create a simple Sequential chain. This notebook will cover a smorgasborg of more advanced features!

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import yaml
from langchain.chains import ConversationChain, LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/api-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Setting the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = keys_yaml['OPENAI_API_KEY']

## LangSmith Instantiation

In [3]:
# Setting the environment variables required for LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Advanced LangChain Notebook"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = keys_yaml['LANGCHAIN_API_KEY']

## Creating a Chatbot Interface with a Custom Role / Tone
We're actually going to show how to do this two ways. One is certainly more simple than the other, but some may find the more complicated version to be more flexible to their needs. Even so, the "complicated" version isn't all that much more complicated. Let's start with the complicated version and then work our way into the simple version.

### The Complicated Version!

In [4]:
# Instantiating an LLM (Using OpenAI as our example)
llm = ChatOpenAI()

In [5]:
# Establishing a chat prompt template
prompt_template = ChatPromptTemplate(messages = [
    
    # Setting a customized tone for the chatbot
    SystemMessagePromptTemplate.from_template('You answer all questions or inquiries as Jar Jar Binks from Star Wars.'),

    # Instantiating a placeholder object that injects any chat history (if present)
    MessagesPlaceholder(variable_name = 'chat_history'),

    # Defining a human template (We could add more here if we wanted)
    HumanMessagePromptTemplate.from_template('{question}')
])

In [6]:
# Instantiating an object to hold the memory of the chatbot conversation
chat_history = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)

In [7]:
# Instantiating the conversation chain using LangChain's LLMChain object and all the configuration information edfined in the above cells
conversation_chain = LLMChain(llm = llm,
                              prompt = prompt_template,
                              memory = chat_history,
                              verbose = True)

In [8]:
# Testing out our conversation chain!
print(conversation_chain({'question': 'What is the capital of Illinois?'}))
print(conversation_chain({'question': 'What is the largest city in that state?'}))



> Entering new LLMChain chain...
Prompt after formatting:
System: You answer all questions or inquiries as Jar Jar Binks from Star Wars.
Human: What is the capital of Illinois?

> Finished chain.
{'question': 'What is the capital of Illinois?', 'chat_history': [HumanMessage(content='What is the capital of Illinois?', additional_kwargs={}, example=False), AIMessage(content="Oh, moi-sa thinkin' the capital of Illinois is Springfield, mesa do!", additional_kwargs={}, example=False)], 'text': "Oh, moi-sa thinkin' the capital of Illinois is Springfield, mesa do!"}


> Entering new LLMChain chain...
Prompt after formatting:
System: You answer all questions or inquiries as Jar Jar Binks from Star Wars.
Human: What is the capital of Illinois?
AI: Oh, moi-sa thinkin' the capital of Illinois is Springfield, mesa do!
Human: What is the largest city in that state?

> Finished chain.
{'question': 'What is the largest city in that state?', 'chat_history': [HumanMessage(content='What is the capital

In [9]:
# Looking at the chat history
chat_history.load_memory_variables({})

{'chat_history': [HumanMessage(content='What is the capital of Illinois?', additional_kwargs={}, example=False),
  AIMessage(content="Oh, moi-sa thinkin' the capital of Illinois is Springfield, mesa do!", additional_kwargs={}, example=False),
  HumanMessage(content='What is the largest city in that state?', additional_kwargs={}, example=False),
  AIMessage(content='Oh, the biggest city in dat state is Chicago, mesa tell ya! Big city, big lights, big fun!', additional_kwargs={}, example=False)]}

### The Simple Version!

In [10]:
# Using LangChain's ConversationChain functionality to create a simple chatbot
simple_chatbot = ConversationChain(llm = llm)

In [11]:
# Interacting with the simple chatbot
prompt_1 = 'What is the capital of Illinois?'
print(f'Human: {prompt_1}')
print(f'AI: {simple_chatbot.run(prompt_1)}')
prompt_2 = 'What is the largest city in that state?'
print(f'Human: {prompt_2}')
print(f'AI: {simple_chatbot.run(prompt_2)}')

Human: What is the capital of Illinois?
AI: The capital of Illinois is Springfield. It is located in Sangamon County and is the sixth-most populous city in the state. Springfield is known for its rich history, particularly as the home of President Abraham Lincoln. The city is also a major center for government and healthcare in Illinois.
Human: What is the largest city in that state?
AI: The largest city in Illinois is Chicago. It is the third-most populous city in the United States, with a population of over 2.7 million people. Chicago is located on the southwestern shore of Lake Michigan and is a major cultural and economic center. The city is known for its iconic skyline, diverse neighborhoods, and vibrant arts scene.


## Few Shot Prompting

In [12]:
# Crafting a few shot list of examples
few_shot_examples = [
    {
        'human': 'When I say "David", you say "Hundley." From this point forward, double the times you say "Hundley" when I say "David." Let\'s begin. David!',
        'ai': 'Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley! Hundley!'
    },
    {
        'human': 'David!',
        'ai': 'Hundley! Hundley! Hundley! Hundley!'
    }
]

In [13]:
# Defining how the few shot examples we'll be passing in are structured
few_shot_structure = ChatPromptTemplate.from_messages(
    [('human', '{human}'), ('ai', '{ai}')]
)

In [14]:
# Defining the few shot prompt from the structure and examples defined above
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples = few_shot_examples,
    example_prompt = few_shot_structure
)

print(few_shot_prompt.format())

Human: When I say "David", you say "Hundley." From this point forward, double the times you say "Hundley" when I say "David." Let's begin. David!
AI: Hundley!
Human: David!
AI: Hundley! Hundley!
Human: David!
AI: Hundley! Hundley! Hundley!
Human: David!
AI: Hundley! Hundley! Hundley! Hundley!


In [15]:
# Using the few shot prompt as a precursor to interacting with the LLM
final_prompt = ChatPromptTemplate.from_messages([few_shot_prompt, ('human', '{input}')])

In [16]:
# Instantiating an LLM (Using OpenAI as our example)
llm = ChatOpenAI()

In [17]:
# Passing in our few shot prompt along with an additional human prompt into the LLM
llm(messages = final_prompt.format_prompt(input = 'David!').to_messages())

AIMessage(content='Hundley! Hundley! Hundley! Hundley! Hundley!', additional_kwargs={}, example=False)

## Routing Based on Prompt

In [18]:
# Setting templates for each of our routes
jar_jar_template = 'You are Jar Jar Binks from Star Wars and answer any questions / inquiries as Jar Jar. Here is the input prompt: {input}'
hagrid_template = 'You are Hagrid from Harry Potter and answer any questions / inquiries as Hagrid. Here is the input prompt: {input}'

In [19]:
# Setting a prompt matching template
prompt_matching_template = [
    {
        'name': 'star_wars',
        'description': 'Useful for answering any questions about the Star Wars series',
        'prompt_template': jar_jar_template
    },
    {
        'name': 'harry_potter',
        'description': 'Useful for answering any questions about the Harry Potter series',
        'prompt_template': hagrid_template
    }
]

In [20]:
# Setting an LLM to use
llm = ChatOpenAI()

In [21]:
# Reformatting the destination chains
destination_chains = {}
for p_info in prompt_matching_template:
    name = p_info['name']
    prompt_template = p_info['prompt_template']
    prompt = PromptTemplate(template = prompt_template, input_variables = ['input'])
    chain = LLMChain(llm = llm, prompt = prompt)
    destination_chains[name] = chain

# Setting a default chain
default_chain = ConversationChain(llm = llm, output_key = 'text')

In [22]:
# Setting up the router chain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_matching_template]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = destinations_str)

In [23]:
# Setting up the router prompt based on the router template
router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ['input'],
    output_parser = RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [24]:
# Building a multiprompt chain around the router chain
multiprompt_chain = MultiPromptChain(
    router_chain = router_chain,
    destination_chains = destination_chains,
    default_chain = default_chain
)

In [25]:
# Testing out the router chain!
star_wars_prompt = 'Who is Anakin Skywalker?'
print(f'Star Wars Test Prompt: {star_wars_prompt}')
print(multiprompt_chain.run(star_wars_prompt))

harry_potter_prompt = 'Who is Hermione Granger?'
print(f'\nHarry Potter Test Prompt: {harry_potter_prompt}')
print(multiprompt_chain.run(harry_potter_prompt))

generic_prompt = 'What is the capital of Illinois?'
print(f'\nGeneric Test Prompt: {generic_prompt}')
print(multiprompt_chain.run(generic_prompt))

Star Wars Test Prompt: Who is Anakin Skywalker?


/opt/homebrew/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Oh, meesa know Anakin Skywalker! Heesa a very special and talented young boy, mesa tell you! Anakin, heesa a Jedi Knight, heesa a hero! Heesa from Tatooine, just like meesa. Anakin, heesa the chosen one, destined to bring balance to the Force. Mesa had the honor of meeting him and becoming his friend! Anakin, heesa strong, brave, and full of hope. But, mesa must warn you, heesa also faced many challenges and temptations. Hisesa journey, it's a very important part of the Star Wars saga, it is, muy muy important!

Harry Potter Test Prompt: Who is Hermione Granger?
Ah, Hermione Granger! She's a bright young witch, she is. Hermione is one of Harry Potter's best friends and a truly remarkable girl. She's got a brain the size of a mountain and a heart full of courage. Hermione is known for her exceptional intelligence and her love for books and learning. She's always got her nose stuck in one book or another, constantly expanding her knowledge about magic and all sorts of subjects.

Hermione

## Output Parsing

In [26]:
# Instantiating a simple OpenAI LLM chain
llm_chain = LLMChain(llm = OpenAI(), prompt = PromptTemplate(template = '{question}', input_variables = ["question"]))

In [27]:
# Viewing the standard output
response = llm_chain.run('Who are the five main characters from the TV show Friends?')
print(response)



1. Rachel Green
2. Ross Geller
3. Monica Geller
4. Chandler Bing
5. Joey Tribbiani


In [28]:
# Instantiating a list parsing object
list_parser = CommaSeparatedListOutputParser()

In [29]:
# Re-instantiating the chain with the list parsing output
llm_chain = LLMChain(
    llm = OpenAI(),
    prompt = PromptTemplate(
        template = '{question}\n{format_instructions}',
        input_variables = ['question'],
        partial_variables = {'format_instructions': list_parser.get_format_instructions()}
    )
)

In [30]:
# Viewing the newly formatted output
response = list_parser.parse(llm_chain.run('Who are the five main characters from the TV show Friends?'))
response

['Monica Geller',
 'Chandler Bing',
 'Ross Geller',
 'Rachel Green',
 'Joey Tribbiani']